In [2]:
import nbimporter
#from wrangling import prep_dataframe
import pandas as pd
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import pydicom
import os
import numpy as np
from torch.autograd import Variable
from torch.nn import Linear, ReLU, CrossEntropyLoss, Sequential, Conv2d, MaxPool2d, Module, Softmax, BatchNorm2d, Dropout
from torch.optim import Adam, SGD
import math
from torch.distributions.normal import Normal
import torch.cuda
import glob

In [3]:
def prep_dataframe(directory, csv):   
    diagnoses = pd.read_csv(csv)
    diagnoses = diagnoses.to_numpy()
    more = np.empty((len(diagnoses), 3), dtype = 'U18')
    for x in range(len(diagnoses)):
        more[x][0] = diagnoses[x][0][3:12]
        more[x][1] = diagnoses[x][0][13:]

    more[:,2] = diagnoses[:,1]

    #test_list = os.listdir(directory)
    test_list = glob.glob("/home/wustl/brooks.kennedy/stage_2_train/*.dcm")[0:100]
    #print(test_list)
    image_arrays = np.zeros([512, 512])
    df = pd.DataFrame(columns = ["Image", "ID", "Diagnoses"])

    for file in test_list:
        if(file != ".DS_Store"):
            try:
                #print("Success with file " + file)
                dataset = pydicom.dcmread(file, force = True)
                diagnoses = more[:,1:][more[:,0]==file[44:53]]
                filtered_diagnoses = diagnoses[:,0][diagnoses[:,1]=='1']
#                any_diagnosis = filtered_diagnoses.size != 0
#                 df = df.append({"Image": dataset.pixel_array, "ID": file[44:53], 
#                                 "Diagnoses": filtered_diagnoses, 
#                                "Hemmorhage": any_diagnosis}, ignore_index = True)
                if filtered_diagnoses.size != 0:
                    df = df.append({"Image": dataset.pixel_array, "ID": file[44:53], 
                                 "Diagnoses": filtered_diagnoses}, ignore_index = True)
            except:
                print("Error with file "+file)  
    return df

In [5]:
train_data

,Image,ID,Diagnoses
0,"[[-1024, -1024, -1024, -1024, -1024, -1024, -1...",a799d9ece,"[intraventricular, subarachnoid, subdural, any]"
1,"[[-1024, -1024, -1024, -1024, -1024, -1024, -1...",a85fed3fc,"[intraparenchymal, any]"
2,"[[-1024, -1024, -1024, -1024, -1024, -1024, -1...",b815b2f2f,"[intraparenchymal, subarachnoid, subdural, any]"
3,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",8b757e6ec,"[intraparenchymal, any]"
4,"[[-2000, -2000, -2000, -2000, -2000, -2000, -2...",0d4ba7df6,"[intraventricular, any]"
5,"[[-2000, -2000, -2000, -2000, -2000, -2000, -2...",8434b7099,"[subarachnoid, any]"
6,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",941750fca,"[intraventricular, any]"
7,"[[-1024, -1024, -1024, -1024, -1024, -1024, -1...",5a210b2bb,"[subdural, any]"
8,"[[152, 152, 155, 159, 160, 158, 154, 154, 153,...",b72f6a4f4,"[subdural, any]"
9,"[[-1024, -1024, -1024, -1024, -1024, -1024, -1...",56ef6e7d2,"[subdural, any]"


In [4]:
train_data = prep_dataframe("/home/wustl/brooks.kennedy/stage_2_train", "/home/wustl/brooks.kennedy/stage_2_train.csv")

In [3]:
#hemmorhage_train_data = train_data.loc[train_data['Hemmorhage']==True]
#train_y = hemmorhage_train_data['Hemmorhage']
#train_x = hemmorhage_train_data['Image']
#ids = hemmorhage_train_data['ID']
train_y = train_data_['Hemmorhage']
train_x = train_data_['Image']
ids = train_data_['ID']

#convert training images, target data, ids to numpy arrays
train_x = train_x.to_numpy()
train_y = train_y.to_numpy()
ids = ids.to_numpy()
for x in range(0, 1000): #filter out incorrectly shaped images
    if (train_x[x].shape != (512, 512)):
        train_x = np.delete(train_x, x)
        train_y = np.delete(train_y, x)
        ids = np.delete(ids, x)
train_x = np.array([np.array(x) for x in train_x])

#convert training images to pytorch tensors
train_x = train_x.reshape(1000, 1, 512, 512)
train_x = train_x.astype('float')
train_x = torch.from_numpy(train_x)

#convert target data to pytorch tensor
train_y = train_y.astype('bool')
train_y = torch.from_numpy(train_y)

This one is non-convolutional:

In [4]:
class Generator(nn.Module):

    def __init__(self):
        super(Generator, self).__init__()       
#         self.cnn_layers = Sequential(
#             #convolution layer 1
#             Conv2d(16, 4, kernel_size = 3, stride=1, padding=1),
#             BatchNorm2d(4),
#             ReLU(inplace=True),
#             MaxPool2d(kernel_size = 2, stride=2),
            
#             #convolution layer 2
#             Conv2d(4, 16, kernel_size = 3, stride=1, padding=1),
#             BatchNorm2d(16),
#             ReLU(inplace=True),
#             MaxPool2d(kernel_size = 2, stride=2),
#         )
        self.dense_layer = Sequential (
            nn.Linear(16, 128),
            nn.Linear(128, 256),
            nn.Linear(256, 262144)
        )
        self.relu = nn.ReLU()
        
    def forward(self, x):
        #x = self.cnn_layers(x)
        #x = self.cnn_layers(x)
        #torch.reshape(x, (4, 4))
        x = self.dense_layer(x)
        x = self.relu(x)
        x = torch.reshape(x, (512, 512))
        return x

In [4]:
channels = 1 # would be 3 for a RGB image
im_size = 512
noise_size = 16

class Generator(nn.Module):
    def _init_(self):
        super(Generator, self)._init_()
        self.layer = Sequential (
            nn.Conv2d(noise_size, im_size*8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(im_size*8),
            nn.ReLU(inplace=True),
            
            nn.Conv2d(im_size*8, im_size*4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(im_size*4),
            nn.ReLU(inplace=True),
            
            nn.Conv2d(im_size*4, im_size*2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(im_size*2),
            nn.ReLU(inplace=True),
            
            nn.Conv2d(im_size*2, im_size, 4, 2, 1, bias=False),
            nn.BatchNorm2d(im_size),
            nn.ReLU(inplace=True),
            
            nn.Conv2d(im_size, channels, 4, 2, 1, bias=False),
            nn.ReLU(inplace=True)
        )
        
    def forward(self, input):
        return self.layer(input)

In [5]:
gen = Generator()
mu = train_x.mean()
std = train_x.std()
noise = torch.normal(mu, std, size=(1,16))
generated_image = gen(noise)

#display the image
generated_image=generated_image.detach().numpy() 
plt.imshow(  generated_image)
generated_image.shape

NameError: name 'train_x' is not defined